In [ ]:
import os
import random
from pathlib import Path

import albumentations as aug
import tensorflow as tf

import train.cityscapes as cityscapes
from train.models import deeplab_v3plus, unet_xception


args_model = "deeplab_v3plus"  # "unet_xception"
args_resize = 64
args_batch = 32
args_augment = False


img_size = (args_resize, args_resize)
batch_size = args_batch
num_classes = 8

augment = None
if args_augment:
    # Image augmentation : None for no augmentation
    augment = aug.Compose(
        [
            aug.OneOf(  # Color augmentations
                [
                    aug.RandomBrightnessContrast(),
                    aug.RandomGamma(),
                    aug.RandomToneCurve(),
                ]
            ),
            aug.OneOf(  # Camera augmentations
                [
                    aug.MotionBlur(),
                    aug.GaussNoise(),
                ]
            ),
            aug.OneOf(  # Geometric augmentations
                [
                    aug.HorizontalFlip(),
                    aug.RandomCrop(
                        width=int(img_size[0] / random.uniform(1.0, 2.0)),
                        height=int(img_size[1] / random.uniform(1.0, 2.0)),
                    ),
                    aug.SafeRotate(
                        limit=15,
                    ),
                ]
            ),
            aug.Resize(
                width=img_size[0],
                height=img_size[1],
            ),
        ]
    )

# Get the model
model_name = f"{args_model}_{args_resize}{'_augment' if args_augment else ''}"
model_path = Path("./../results/", model_name)


try:
    model = tf.keras.models.load_model(
        Path(model_path, "model"),
        custom_objects={
            "UpdatedMeanIoU": cityscapes.UpdatedMeanIoU,
            "jaccard_loss": cityscapes.jaccard_loss,
        },
    )
    print(f">>> Loaded model {model_name} .")
except Exception as e:
    print(f">>> Error loading model {model_name} : {e}")
    print(f">>> Creating new model {model_name}.")

    model = None
    if args_model == "unet_xception":
        model = unet_xception.get_model(
            img_size, num_classes, model_name=model_name
        )
    elif args_model == "deeplab_v3plus":
        model = deeplab_v3plus.get_model(
            weights="cityscapes",
            input_tensor=None,
            input_shape=(args_resize, args_resize, 3),
            classes=8,
            backbone="xception", # "mobilenetv2"
            OS=16,
            alpha=1.0,
            activation="softmax",
            model_name=model_name,
        )

model.summary()

# Configure the model for training.
model.compile(
    optimizer="adam",
    loss=cityscapes.jaccard_loss,  # "sparse_categorical_crossentropy",
    metrics=[
        cityscapes.UpdatedMeanIoU(
            name="MeanIoU", num_classes=num_classes
        ),  # https://ilmonteux.github.io/2019/05/10/segmentation-metrics.html
    ],
)

raw_data_path = Path("./../data/raw")
leftImg8bit_path = Path(raw_data_path, "leftImg8bit")
gtFine_path = Path(raw_data_path, "gtFine")

# Train dataset
train_input_img_paths = sorted(
    Path(leftImg8bit_path, "train").glob("**/*_leftImg8bit.png")
)
train_label_ids_img_paths = sorted(
    Path(gtFine_path, "train").glob("**/*_gtFine_labelIds.png")
)

# Validation dataset
val_input_img_paths = sorted(
    Path(leftImg8bit_path, "val").glob("**/*_leftImg8bit.png")
)
val_label_ids_img_paths = sorted(
    Path(gtFine_path, "val").glob("**/*_gtFine_labelIds.png")
)
val_label_colors_img_paths = sorted(
    Path(gtFine_path, "val").glob("**/*_color.png")
)


# train model
model.fit(
    cityscapes.CityscapesGenerator(
        batch_size,
        img_size,
        train_input_img_paths,
        train_label_ids_img_paths,
        augment,
    ),
    validation_data=cityscapes.CityscapesGenerator(
        batch_size,
        img_size,
        val_input_img_paths,
        val_label_ids_img_paths,
        augment,
    ),
    epochs=100,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(
            patience=2,
            factor=0.5,
            min_delta=1e-2,
            min_lr=1e-7,
            verbose=1,
        ),
        tf.keras.callbacks.EarlyStopping(
            patience=5,
            restore_best_weights=True,
            min_delta=1e-3,
            verbose=1,
        ),
        tf.keras.callbacks.TensorBoard(log_dir=Path(model_path, "./logs")),
        cityscapes.CityscapesViewerCallback(
            val_input_img_paths, val_label_colors_img_paths, img_size
        ),
    ],
)

# save model
model.save(Path(model_path, "model"))
